[![Fixel Algorithms](https://fixelalgorithms.co/images/CCExt.png)](https://fixelalgorithms.gitlab.io/)

# Machine Learning Methods

## Supervised Learning - Ensemble Methods - Random Forests

> Notebook by:
> - Royi Avital RoyiAvital@fixelalgorithms.com

## Revision History

| Version | Date       | User        |Content / Changes                                                   |
|---------|------------|-------------|--------------------------------------------------------------------|
| 0.1.000 | 19/02/2023 | Royi Avital | First version                                                      |
|         |            |             |                                                                    |

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/FixelAlgorithmsTeam/FixelCourses/blob/master/MachineLearningMethods/2023_01/0031EnsembleRandomForests.ipynb)

In [ ]:
# Import Packages

# General Tools
import numpy as np
import scipy as sp
import pandas as pd

# Machine Learning
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.inspection import permutation_importance
from sklearn.model_selection import train_test_split

# Miscellaneous
import os
from platform import python_version
import random

# Typing
from typing import Callable, List, Tuple

# Visualization
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm, Normalize
import seaborn as sns
# from bokeh.plotting import figure, show

# Jupyter
from IPython import get_ipython
from IPython.display import Image, display
from ipywidgets import Dropdown, FloatSlider, interact, IntSlider, Layout

## Notations

* <font color='red'>(**?**)</font> Question to answer interactively.
* <font color='blue'>(**!**)</font> Simple task to add code for the notebook.
* <font color='green'>(**@**)</font> Optional / Extra self practice.
* <font color='brown'>(**#**)</font> Note / Useful resource / Food for thought.

In [ ]:
# Configuration
%matplotlib inline

seedNum = 512
np.random.seed(seedNum)
random.seed(seedNum)

# sns.set_theme() #>! Apply SeaBorn theme

runInGoogleColab = 'google.colab' in str(get_ipython())

In [ ]:
# Constants

FIG_SIZE_DEF    = (8, 8)
ELM_SIZE_DEF    = 50
CLASS_COLOR     = ('b', 'r')
EDGE_COLOR      = 'k'
MARKER_SIZE_DEF = 10
LINE_WIDTH_DEF  = 2


In [ ]:
# Fixel Algorithms Packages


## Random Forests Classification

In this note book we'll use the Random Forests based classifier in the task of estimating whether a passenger on the Titanic will or will not survive.  
We'll focus on basic pre processing of the data and analyzing the importance of features using the classifier.

* <font color='brown'>(**#**)</font> This is a very popular data set for classification. You may have a look for notebooks on teh need with a deeper analysis of the data set itself.

In [ ]:
# Parameters

# Data
trainRatio = 0.75

# Model
numEst = 150
spliCrit = 'gini'
maxLeafNodes = 20
outBagScore = True

# Feature Permutation
numRepeats = 50


In [ ]:
# Auxiliary Functions

def PlotRegResults( vY, vYPred, hA:plt.Axes = None, figSize: Tuple[int, int] = FIG_SIZE_DEF, lineWidth: int = LINE_WIDTH_DEF, elmSize: int = ELM_SIZE_DEF, classColor: Tuple[str, str] = CLASS_COLOR, axisTitle: str = None ) -> plt.Axes:

    if hA is None:
        hF, hA = plt.subplots(figsize = figSize)
    else:
        hF = hA.get_figure()

    numSamples = len(vY)
    if (numSamples != len(vYPred)):
        raise ValueError(f'The inputs `vY` and `vYPred` must have the same number of elements')
    
    
    hA.plot(vY, vY, color = 'r', lw = lineWidth, label = 'Ground Truth')
    hA.scatter(vY, vYPred, s = elmSize, color = classColor[0], edgecolor = 'k', label = f'Estimation')
    hA.set_xlabel('Label Value')
    hA.set_ylabel('Prediction Value')
    # hA.axis('equal')
    if axisTitle is not None:
        hA.set_title(axisTitle)
    hA.legend()
    
    return hA


## Generate / Load Data

In this section we'll load the data.  
We'll create a Data Frame of the data and later will separate it into features and labels.


In [ ]:
# Loading / Generating Data

dfX, dsY = fetch_openml('titanic', version = 1, return_X_y = True, as_frame = True, parser = 'auto')

print(f'The data shape: {dfX.shape}')

In [ ]:
dfX.head(10)

In [ ]:
# Dropping Features
# We'll drop the `name` and `ticket` features (Avoid 1:1 identification)

dfX = dfX.drop(columns = ['name', 'ticket'])

* <font color='brown'>(**#**)</font> Pay attention that we dropped the features, but a deeper analysis could extract information from them. 

In [ ]:
dfX

In [ ]:
dsY.head(10)

## Pre Processing and EDA

In [ ]:
dfData = pd.concat([dfX, dsY], axis = 1)

In [ ]:
dfData.info()

In [ ]:
# Null / NA / NaN Matrix

#===========================Fill This===========================#
# 1. Calculate the logical map of invalid values using the method `isna()`.
mInvData = dfData.isna() #<! The logical matrix of invalid values
#===============================================================#

hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
sns.heatmap(data = mInvData, square = False, ax = hA)
hA.set_title('Invalid Data Map')

plt.show()

Since the features `cabin`, `boat`, `body` and `home.dest` have mostly non valid value we'll drop them as well.

In [ ]:
# Removing Features with Invalid Values

dfData = dfData.drop(columns = ['cabin', 'boat', 'body', 'home.dest'])

In [ ]:
# Data Visualization
# Basic EDA on the Data

numCol = dfData.shape[1]
lCols  = dfData.columns
numAx  = int(np.ceil(np.sqrt(numCol)))

hIsCatLikData = lambda dsX: (pd.api.types.is_categorical_dtype(dsX) or pd.api.types.is_bool_dtype(dsX) or pd.api.types.is_object_dtype(dsX) or pd.api.types.is_integer_dtype(dsX))

hF, hAs = plt.subplots(nrows = numAx, ncols = numAx, figsize = (16, 12))
hAs = hAs.flat

for ii in range(numCol):
    colName = dfData.columns[ii]
    if hIsCatLikData(dfData[colName]):
        sns.histplot(data = dfData, x = colName, hue = 'survived', stat = 'count', discrete = True, common_norm = True, multiple = 'dodge', ax = hAs[ii])
    else:
        sns.kdeplot(data = dfData, x = colName, hue = 'survived', fill = True, common_norm = True, ax = hAs[ii])

plt.show()

* <font color='red'>(**?**)</font> Have a look on the features and try to estimate their importance for the estimation.

### Filling Missing Data

For the rest of the missing values we'll use a simple method of interpolation:

 - Categorical Data: Using the mode value.
 - Numeric Data: Using the median / mean.

* <font color='brown'>(**#**)</font> We could employ much more delicate and sophisticated data. For instance, use the mean value of the same `pclass`. Namely profiling the data by other features to interpolate the missing feature.

In [ ]:
# Missing Value by Dictionary
dNaNs = {'embarked': dfData['embarked'].mode()[0], 'age': dfData['age'].median(), 'fare': dfData['fare'].median()}

dfData = dfData.fillna(value = dNaNs, inplace = False) #<! We can use the `inplace` for efficiency

In [ ]:
# Null / NA / NaN Matrix

mInvData = dfData.isna() #<! The logical matrix of invalid values

hF, hA = plt.subplots(figsize = FIG_SIZE_DEF)
sns.heatmap(data = mInvData, square = False, ax = hA)
hA.set_title('Invalid Data Map')

plt.show()

### Conversion of Categorical Data

In this notebook we'll use the `RandomForestClassifier` implementation of Random Forests.  
At the moment, it doesn't support Categorical Data, hence we'll use Dummy Variables (One Hot Encoding).

* <font color='brown'>(**#**)</font> Pay attention that one hot encoding is an inferior approach to having a real support for categorical data.
* <font color='brown'>(**#**)</font> For 2 values categorical data (Binary feature) there is no need fo any special support.

In [ ]:
# Features Encoding
# 1. The feature 'embarked' -> One Hot Encoding.
# 1. The feature 'sex' -> Mapping (Female -> 0, Male -> 1).
dfData = pd.get_dummies(dfData, columns = ['embarked'], drop_first = False)
dfData['sex'] = dfData['sex'].map({'female': 0, 'male': 1})

In [ ]:
# Convert Data Type
dfData = dfData.astype(dtype = {'pclass': np.uint8, 'sex': np.uint8, 'sibsp': np.uint8, 'parch': np.uint8, 'survived': np.uint8})

### Split Train & Test

In [ ]:
dfX = dfData.drop(columns = ['survived'])
dsY = dfData['survived']

print(f'The features data shape: {dfX.shape}')
print(f'The labels data shape: {dsY.shape}')

In [ ]:
# Split the Data

dfXTrain, dfXTest, dsYTrain, dsYTest = train_test_split(dfX, dsY, train_size = trainRatio, random_state = seedNum, shuffle = True, stratify = dsY)



## Train the Random Forests Model

The random forests models basically creates weak classifiers by limiting their access to the data and features.  
This basically also limits their correlation which means we can use their mean in order to reduce the variance of the estimation.

In [ ]:
# Construct the Model & Train

oRndForestsCls = RandomForestClassifier(n_estimators = numEst, criterion = spliCrit, max_leaf_nodes = maxLeafNodes, oob_score = outBagScore)
oRndForestsCls = oRndForestsCls.fit(dfXTrain, dsYTrain)

In [ ]:
# Scores of the Model

print(f'The train accuracy     : {oRndForestsCls.score(dfXTrain, dsYTrain):0.2%}')
print(f'The out of bag accuracy: {oRndForestsCls.oob_score_:0.2%}')
print(f'The test accuracy      : {oRndForestsCls.score(dfXTest, dsYTest):0.2%}')

* <font color='blue'>(**!**)</font> Try different values for the model's hyper parameter (Try defaults as well).

## Feature Importance Analysis

### Contribution to Impurity Decrease

This is the default method for feature importance of decision trees based methods.  
It basically sums the amount of impurity reduced by splits by each feature.

In [ ]:
# Extract Feature Importance
vFeatImp = oRndForestsCls.feature_importances_
vIdxSort = np.argsort(vFeatImp)



In [ ]:
# Display Feature Importance
hF, hA = plt.subplots(figsize = (16, 8))
hA.bar(x = dfXTrain.columns[vIdxSort], height = vFeatImp[vIdxSort])
hA.set_title('Features Importance of the Model')
hA.set_xlabel('Feature Name')
hA.set_title('Importance')

plt.show()

* <font color='red'>(**?**)</font> Do we need all 3: `embarked_C`, `embarked_Q` and `embarked_S`? Look at the options of `get_dummies()`.

### Permutation Effect

This is a more general method to measure the importance of a feature.  
We basically replace the values with "noise" to see how much performance has been deteriorated.

* <font color='brown'>(**#**)</font> This is highly time consuming operation. Hence the speed of decision trees based methods creates a synergy.

In [ ]:
# Test Permutations
oFeatImpPermTrain = permutation_importance(oRndForestsCls, dfXTrain, dsYTrain, n_repeats = numRepeats)
oFeatImpPermTest  = permutation_importance(oRndForestsCls, dfXTest, dsYTest, n_repeats = numRepeats)

In [ ]:
# Generate Data Frame

dT = {'Feature': [], 'Importance': [], 'Data': []}

for (dataName, mScore) in [('Train', oFeatImpPermTrain.importances), ('Test', oFeatImpPermTest.importances)]:
    for ii, featName in enumerate(dfX.columns):
        for jj in range(numRepeats):
            dT['Feature'].append(featName)
            dT['Importance'].append(mScore[ii, jj])
            dT['Data'].append(dataName)

dfFeatImpPerm = pd.DataFrame(dT)
dfFeatImpPerm

In [ ]:
# Plot Results
hF, hA = plt.subplots(figsize = (12, 8))
sns.boxplot(data = dfFeatImpPerm, x = 'Feature', y = 'Importance', hue = 'Data', ax = hA)

plt.show()

* <font color='green'>(**@**)</font> Try extracting better results by using the dropped features.